# Generating music

#### Importing liabraries

In [1]:
from music21 import *
import pickle
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


#### loading notes

In [2]:
with open('data/notes.pickle', 'rb') as f:
    notes = pickle.load(f)

pitchNames = set (notes)
nNotes = len (pitchNames)
pitchNames = sorted (pitchNames)

#### Encoding notes

In [3]:
noteCoding = {}

code = 0
for i in pitchNames :
    noteCoding [i] = code
    code += 1

#### One hot encoding

In [4]:
sequence = 100
nInput = []
nOutput = []

for i in range (0, len (notes) - sequence, 1) : 
    
    sIn = notes [i : i + sequence]
    sOut = notes [i + sequence]
    nInput.append ([noteCoding [c]for c in sIn])
    nOutput.append (noteCoding [sOut])

In [5]:
nInput = np.reshape (nInput, (len (nInput), sequence, 1))

In [6]:
nOutput = to_categorical (nOutput)

#### Model

In [7]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(nInput.shape[1], nInput.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.25))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(nNotes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
 model.load_weights('weights/singleTrack/weights-improvement-69-0.7378-bigger.hdf5')

#### Generating random inputs

In [9]:
s = np.random.randint (0, len (notes))
inNotes = list (np.ravel (nInput[s]))

outputElements = []

noteCoding = dict((number, note) for number, note in enumerate (pitchNames))

#### Generating notes

In [10]:
for i in range (100) :
    
    arrayInput = np.array (inNotes)
    netIn = np.reshape (arrayInput, (1, sequence, 1)) / float (nNotes)
    
    prediction = np.argmax (model.predict (netIn, verbose = 1))
    
    inNotes = inNotes [1:100]
    inNotes.append (prediction)
    
    outputElements.append (noteCoding [prediction])

print (outputElements)

1/1 [==============================] - 0s 153ms/step
['3.5.10', 'D4', 'E-4', '5.7.0', 'F2', 'G#4', '3.5.10', 'C#5', 'E-5', 'C#5', 'C5', 'B-4', '5.7.0', 'C#5', 'C5', 'B-2', 'G#4', 'E-3', 'C3', 'G4', 'F2', 'F4', '3.5.10', 'G4', 'G#4', '5.7.0', 'F2', 'C#5', '3.5.10', 'F#5', 'G#5', 'F#5', 'F5', '5.7.0', 'B-2', 'E-3', 'C3', 'F2', 'A4', '3.5.10', 'D5', 'E5', 'D5', 'C#5', '5.7.0', 'F2', '3.5.10', '5.7.0', 'B-2', 'E-3', 'C3', 'F2', '3.5.10', '5.7.0', 'C#4', 'F2', 'E-4', 'F4', 'B-5', 'G#4', 'G#4', 'G#4', 'G5', 'G5', 'G#5', 'G5', 'G5', 'G5', 'B-4', 'G5', 'B-4', 'B-5', 'F2', 'C3', 'G#4', 'E-5', 'G#4', 'F2', 'C3', 'C#5', 'C#5', 'C#5', 'C#4', 'E-3', 'E-4', 'C#4', 'C#5', 'E-3', 'B2', 'F#2', 'G#4', 'B2', 'G#5', 'F#2', 'B3', 'G#4', 'D3', 'B2', 'E4', 'B2']


#### Converting notes to music file

In [11]:
outputMusic = []
offset = 0

for ele in outputElements : 
    
    if ('.' in ele) or ele.isdigit () :
        n = []
        
        for i in ele.split ('.') : 
            temp = note.Note (int (i))
            temp.storedInstrument = instrument.Piano ()
            n.append (temp)
        
        temp = chord.Chord (n)
        temp.offset = offset
        
        outputMusic.append (temp)
        
    else : 
        temp = note.Note (ele)
        temp.storedInstrument = instrument.Piano ()
        temp.offset = offset
        
        outputMusic.append (temp)
        
    offset += 0.5
        
outputMusic = stream.Stream (outputMusic)
outputMusic.write ('midi', fp = 'output/singletrack/music.mid')

'output/singletrack/music.mid'

#### Music Generated